# Feature Selection - Random Forest

## Libraries and Configurations

Import configuration files

In [8]:
from configparser import ConfigParser

config = ConfigParser()
config.read("../config.ini")

['../config.ini']

Import **data libraries**

In [9]:
import pandas as pd

Import **ML libraries**

In [ ]:
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassfier
from sklearn.feature_selection import SelectFromModel

Import **other libraries**

In [10]:
from rich.progress import Progress
from rich import traceback

traceback.install()

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x10e9b8dd0>>

Custom helper scripts

In [11]:
%cd ..
from scripts import plotHelper, encodingHelper
%cd data_exploration_cleaning

/Users/bacci/Library/CloudStorage/SynologyDrive-giovanni/Research 🌱/Repositories/COMPACT/notebooks
/Users/bacci/Library/CloudStorage/SynologyDrive-giovanni/Research 🌱/Repositories/COMPACT/notebooks/data_exploration_cleaning


## SUM Encoded Data

In [12]:
# Combined dataframe
df_sum_csv = config["DEFAULT"]["interim_path"] + "encoded_sum_balanced_df.csv"

In [13]:
df_sum = pd.read_csv(df_sum_csv, index_col=0)

In [14]:
df_sum

,Timestamp,MAC Address,Channel,DS Channel,HT Capabilities SUM,Extended Capabilities SUM,Vendor Specific Tags SUM,Supported Rates SUM,Extended Supported Rates SUM,Length,Label
0,2023-05-20 13:52:01.864465952,d2:6b:aa:b5:fb:ed,1,1.0,2709,830,-1,479,915,135,iPhone12Pro_C
1,2023-05-20 13:52:01.884716034,d2:6b:aa:b5:fb:ed,1,1.0,2709,830,-1,479,915,135,iPhone12Pro_C
2,2023-05-20 13:52:01.910542011,d2:6b:aa:b5:fb:ed,6,6.0,2709,830,-1,479,915,135,iPhone12Pro_C
3,2023-05-20 13:52:01.930788994,d2:6b:aa:b5:fb:ed,6,6.0,2709,830,-1,479,915,135,iPhone12Pro_C
4,2023-05-20 13:52:01.968745947,d2:6b:aa:b5:fb:ed,11,11.0,2709,830,-1,479,915,135,iPhone12Pro_C
...,...,...,...,...,...,...,...,...,...,...,...
13939,2021-07-07 12:02:57.579541922,da:a1:19:00:17:f9,6,1.0,2663,1032,747,447,915,182,XiaomiRedmiNote7_S
13940,2021-07-07 11:32:04.533828019,da:a1:19:1a:cc:8f,6,8.0,2663,1032,747,447,915,198,XiaomiRedmiNote7_S
13941,2021-07-07 11:46:50.089955091,da:a1:19:41:c9:b1,11,5.0,2663,1032,747,447,915,143,XiaomiRedmiNote7_S
13942,2021-07-07 12:16:31.309731960,da:a1:19:c7:24:b1,1,3.0,2663,1032,747,447,915,182,XiaomiRedmiNote7_S


### Split dataframe

In [ ]:
X_train, y_train, X_test, y_test = model_selection.train_test_split(
    df_sum, test_size=0.3, random_state=42
)

In [ ]:
sel = SelectFromModel(RandomForestClassifier(n_estimators=100))
sel.fit(X_train, y_train)